In [1]:
from keras.models import load_model
import pandas as pd
import pickle
import tensorflow as tf
import numpy as np
import keras
import matplotlib.pyplot as plt
import shap
import seaborn as sns
import os
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve, accuracy_score

In [2]:
## List of models
all_models = [f for f in os.listdir("../../models/TopoDNN/trained_models/")]
print("\n".join(all_models))

topodnnmodel_v3
topodnnmodel_v4
topodnnmodel_standardize_pt
topodnnmodel_30
topodnnmodel_v1
topodnnmodel_v2
topodnnmodel_pt
topodnnmodel
topodnnmodel_pt0


In [3]:
def eval(model, mode, df_test):
    if mode == '_pt0':
        df_test = df_test.loc[:, df_test.columns != 'pt_0']
    elif mode == '_pt':
        pt_cols = [col for col in df_test.columns if 'pt' in col]
        df_test = df_test.drop(pt_cols, axis=1)
    elif mode == '_30':
        df_test = df_test.iloc[:,:30]
    elif mode == '_standardize_pt':
        pt_cols = [col for col in df_test.columns if 'pt' in col]
        df_test[pt_cols] = (df_test[pt_cols] - df_test[pt_cols].mean())/df_test[pt_cols].std()
    feats = np.array(df_test)
    data = np.copy(feats)
    return model(data)

In [7]:
for model_name in all_models:
    #Loading data
    df_test = pd.read_pickle("../../datasets/topoprocessed/test.pkl")
    labels = np.array(df_test.loc[:,df_test.columns == 'is_signal_new']).reshape(-1)
    df_test = df_test.loc[:,df_test.columns != 'is_signal_new']
    mode = model_name.replace('topodnnmodel', '')
    #Load model
    model = load_model("../../models/TopoDNN/trained_models/"+model_name)
    results = np.array(eval(model, mode, df_test)).flatten()
    auc = roc_auc_score(labels, results)*100
    results[results >= 0.5] = 1
    results[results < 0.5] = 0
    acc = accuracy_score(labels, results)*100
    print("{} \t ROC-AUC: {:.4f}%, Accuracy: {:.4f}%".format(model_name,auc,acc))

topodnnmodel_v3 	 ROC-AUC: 97.1243%, Accuracy: 91.5488%
topodnnmodel_v4 	 ROC-AUC: 97.1985%, Accuracy: 91.6295%
topodnnmodel_standardize_pt 	 ROC-AUC: 97.1045%, Accuracy: 91.5114%
topodnnmodel_30 	 ROC-AUC: 96.2256%, Accuracy: 89.4649%
topodnnmodel_v1 	 ROC-AUC: 97.1514%, Accuracy: 91.5614%
topodnnmodel_v2 	 ROC-AUC: 97.1655%, Accuracy: 91.6267%
topodnnmodel_pt 	 ROC-AUC: 95.8363%, Accuracy: 89.6710%
topodnnmodel 	 ROC-AUC: 97.1203%, Accuracy: 91.5547%
topodnnmodel_pt0 	 ROC-AUC: 97.0128%, Accuracy: 91.4228%
